## Exercises I
### Exercise1:
Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

In [1]:
import pandas as pd
import math
import numpy as np
from env import host, username, password
from pydataset import data

### Exercise 2:
cd into your exercises folder for this module and run echo env.py >> .gitignore

### Exercise 3:
Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [2]:
def get_db_url(username, host, password, db_name):
    url = f'mysql+pymysql://{username}:{password}@{host}/{db_name}'
    return url

### Exercise 4:
Use your function to obtain a connection to the employees database.

In [2]:
#get_db_url(username,host,password,'employees')

### Exercise 5A:
Intentionally make a typo in the database url. What kind of error message do you see?

In [25]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
LIMIT 100
'''

employees = pd.read_sql(sql, (get_db_url(username,host,password, 'employees')))
employees.head()
# 'Access denied due to trying to access a non-existent db'

,emp_no,first_name,last_name
0,10001,Georgi,Facello
1,10002,Bezalel,Simmel
2,10003,Parto,Bamford
3,10004,Chirstian,Koblick
4,10005,Kyoichi,Maliniak


### Exercise 5B:
Intentionally make an error in your SQL query. What does the error message look like?

In [24]:
employees = pd.read_sql(sql, (get_db_url(username,host,password, 'employees')))
employees.head()

# 'Unknown column'

,emp_no,first_name,last_name
0,10002,Bezalel,Simmel
1,10006,Anneke,Preusig
2,10007,Tzvetan,Zielinski
3,10009,Sumant,Peac
4,10010,Duangkaew,Piveteau


### Exercise 6:
Read the employees and titles tables into two separate DataFrames

In [44]:
sql = '''
SELECT
    *
FROM employees
LIMIT 100
'''


employees = pd.read_sql(sql, (get_db_url(username,host,password, 'employees')))

sql = '''
SELECT
    *
FROM titles
LIMIT 100
'''
titles = pd.read_sql(sql,(get_db_url(username,host,password, 'employees')))


### Exercise 7:
How many rows and columns do you have in each DataFrame? Is that what you expected?

In [45]:
employees.info()
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   emp_no      100 non-null    int64 
 1   birth_date  100 non-null    object
 2   first_name  100 non-null    object
 3   last_name   100 non-null    object
 4   gender      100 non-null    object
 5   hire_date   100 non-null    object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   emp_no     100 non-null    int64 
 1   title      100 non-null    object
 2   from_date  100 non-null    object
 3   to_date    100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


### Exercise 8:
Display the summary statistics for each DataFrame.

In [46]:
print(employees.describe())
print(titles.describe())

             emp_no
count    100.000000
mean   10050.500000
std       29.011492
min    10001.000000
25%    10025.750000
50%    10050.500000
75%    10075.250000
max    10100.000000
             emp_no
count    100.000000
mean   10034.320000
std       18.656859
min    10001.000000
25%    10018.000000
50%    10035.500000
75%    10050.000000
max    10066.000000


### Exercise 9:
How many unique titles are in the titles DataFrame?

In [48]:
titles.title.unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader'], dtype=object)

### Exercise 10:
What is the oldest date in the to_date column?

In [51]:
titles.to_date.min()

datetime.date(1987, 1, 27)

### Exercise 11:
What is the most recent date in the to_date column?

In [56]:
titles.to_date.max()

datetime.date(9999, 1, 1)

## Exercises II
### Exercise 1:
Copy the users and roles DataFrames from the examples above.

In [59]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### Exercise 2:
What is the result of using a right join on the DataFrames?

In [61]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}))


,id,employee,role_id,role
0,1.0,bob,1,admin
1,2.0,joe,2,author
2,3.0,sally,3,reviewer
3,4.0,adam,3,reviewer
4,NaN,NaN,4,commenter


### Exercise 3:
What is the result of using an outer join on the DataFrames?

In [62]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer'))


,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


### Exercise 4:
What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [66]:
users.merge(roles, 
            left_on= 'role_id', 
            right_on = 'id', 
            how='inner').drop(columns=['role_id','id_x','id_y'])

,name_x,name_y
0,bob,admin
1,joe,author
2,sally,reviewer
3,adam,reviewer


### Exercise 5:
Load the mpg dataset from PyDataset.

In [3]:
mpg = data('mpg')

### Exercise 6:
Output and read the documentation for the mpg dataset.

In [72]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


### Exercise 7:
How many rows and columns are in the dataset?

In [74]:
mpg.shape

(234, 11)

### Exercise 8:
Check out your column names and perform any cleanup you may want on them.

In [35]:
mpg = mpg.rename(columns ={
    'displ':'liters',
    'cyl':'cylinder',
    'trans':'transmission',
    'drv':'drive',
    'fl':'fuel',
    'cty':'city',
    'hwy':'highway'})

### Exercise 9:
Display the summary statistics for the dataset.

In [5]:
mpg.describe()

,liters,year,cylinder,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


### Exercise 10:
How many different manufacturers are there?

In [18]:
len(mpg.manufacturer.unique())

15

### Exercise 11:
How many different models are there?

In [19]:
len(mpg.model.unique())

38

### Exercise 12:
Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [22]:
mpg['mileage_difference'] = (mpg.highway - mpg.city)

### Exercise 13:
Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [25]:
mpg['average_mileage'] = (mpg.highway + mpg.city)/2

### Exercise 14:
Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [40]:
mpg['is_automatic'] = mpg.transmission.str.startswith('a')

,manufacturer,model,liters,year,cylinder,transmission,drive,city,highway,fuel,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


### Exercise 15:
Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [64]:
mpg.groupby('manufacturer').average_mileage.mean().nlargest(n=1)

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

### Exercise 16:
Do automatic or manual cars have better miles per gallon?

In [61]:
mpg.groupby('is_automatic').average_mileage.mean()

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

## Exercises III
### Exercise 1:
Use your get_db_url function to help you explore the data from the chipotle database.

In [15]:
sql = 'SELECT * FROM orders'
chipotle = pd.read_sql(sql, (get_db_url(username,host,password, 'chipotle')))



### Exercise 2:
What is the total price for each order?

In [29]:
chipotle['item_price'] = chipotle.item_price.str.strip('$').astype(float)


In [5]:
chipotle.groupby('order_id').agg('sum')

,id,quantity,item_price
order_id,,,
1,10,4,11.56
2,5,2,16.98
3,13,2,12.67
4,17,2,21.00
5,21,2,13.70
...,...,...,...
1830,9223,2,23.00
1831,13842,3,12.90
1832,9233,2,13.20


### Exercise 3:
What are the most popular 3 items?

In [16]:
chipotle.groupby(['item_name']).quantity.sum().nlargest(n=3)

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

### Exercise 4:
Which item has produced the most revenue?

In [31]:
chipotle.groupby(['item_name']).item_price.sum().nlargest(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

### Exercise 5:
Join the employees and titles DataFrames together.

In [46]:
employees = pd.read_sql('select * from employees', (get_db_url(username,host,password, 'employees')))
titles = pd.read_sql('select * from titles', (get_db_url(username,host,password, 'employees')))

employees = employees.merge(titles,
               left_on='emp_no',
               right_on='emp_no',
               how='inner')


### Exercise 6:
For each title, find the hire date of the employee that was hired most recently with that title.

In [51]:
employees.groupby(['title']).hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

### Exercise 7:
Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [4]:
emp = pd.read_sql('''select * from departments as d
              join dept_emp using (dept_no)
              join titles using (emp_no)
              where titles.to_date > now()
              and dept_emp.to_date > now()''' ,(get_db_url(username,host,password, 'employees')) )
pd.crosstab(emp.title, emp.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1652,0,0,0,1402,389,77,0
Engineer,627,14040,0,0,0,12081,3405,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,38816,0,0,0,33625,9458,2250,0
Senior Staff,11268,1085,9545,9824,11290,1123,0,9092,28797
Staff,3574,315,2891,3073,3551,349,0,2870,8903
Technique Leader,241,5477,0,0,0,4723,1293,321,0
